# Step 1: .INI File Configuration

In [1]:
import configparser
import os
vAR_read_config = configparser.ConfigParser(allow_no_value=True)
vAR_INI_FILE_PATH = os.environ.get('PATIENT_SEGMENTATION')
vAR_read_config.read(vAR_INI_FILE_PATH)

['C:\\Users\\YOGESH\\Desktop\\AI\\PATIENT SEGMENTATION\\DL\\INI\\PATIENT SEGMENTATION.ini']

# Step 2: Importing libraries for Patient Segmentation

In [7]:
from fastai import *
from fastai.vision import *
from fastai.layers import MSELossFlat, CrossEntropyFlat
from torchvision import transforms
import warnings
warnings.filterwarnings("ignore")
import csv
from os import listdir
from os.path import isfile, join
import ast



# Step 3: Setting variables

In [9]:
vAR_TESTDATA=vAR_read_config['PATH']['vAR_TESTDATA']
vAR_MODELOUTCOME=vAR_read_config['PATH']['vAR_MODELOUTCOME']
vAR_DICT_GENDER=ast.literal_eval(vAR_read_config['PATH']['vAR_DICT_GENDER'])
vAR_DICT_ETHNICITY=ast.literal_eval(vAR_read_config['PATH']['vAR_DICT_ETHNICITY'])

# Step 3: Getting list of all files in test folder


In [10]:
vAR_FILENAME = [vAR_F for vAR_F  in listdir(vAR_TESTDATA) if isfile(join(vAR_TESTDATA, vAR_F))]

# Step 4: Patient segmentation function definiton

In [16]:
class AgenethPredictor():
    def __init__(self, model):
        self.model = model
        self.tfms = get_transforms()[1]
        self.norm = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #imagenet stats
        self.gender = vAR_DICT_GENDER
        self.ethnicity = vAR_DICT_ETHNICITY

    def predict(self,x):
        #x is a PIL Image
        vAR_X = Image(pil2tensor(x, dtype=np.float32).div_(255))
        vAR_X = vAR_X.apply_tfms(self.tfms, size = 64)
        vAR_X = self.norm(vAR_X.data)
        vAR_PREDS = self.model(vAR_X.unsqueeze(0))
        vAR_AGE = int(torch.exp(vAR_PREDS[0]*4.75).item())
        vAR_GENDER = self.gender[torch.softmax(vAR_PREDS[1],1).argmax().item()]
        vAR_ETHNICITY = self.ethnicity[torch.softmax(vAR_PREDS[2],1).argmax().item()]
        return vAR_AGE, vAR_GENDER, vAR_ETHNICITY

In [17]:
class MultiTaskModel(nn.Module):
    def __init__(self, arch,ps=0.5):
        super(MultiTaskModel,self).__init__()
        self.encoder = create_body(arch)
        self.fc1 = create_head(1024,1,ps=ps)
        self.fc2 = create_head(1024,2,ps=ps)
        self.fc3 = create_head(1024,5,ps=ps)

    def forward(self,x):

        x = self.encoder(x)
        age = torch.sigmoid(self.fc1(x))
        gender = self.fc2(x)
        ethnicity = self.fc3(x)

        return [age, gender, ethnicity]

# Step 5: Downloading pre-trained model

In [18]:
model = MultiTaskModel(models.resnet34)

# Step 6: Loading Trained Weigths

In [19]:
model.load_state_dict(torch.load('finalmodel'))

<All keys matched successfully>

In [20]:
ageneth_predictor = AgenethPredictor(model)

In [21]:
model.eval()

MultiTaskModel(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

# Step 7: Patient Segmentation and writing in .CSV file

In [26]:
with open(vAR_MODELOUTCOME+'\\'+'PATIENT SEGMENTATION.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", 'Age', 'Gender', 'Race']) 


In [27]:
for i in vAR_FILENAME:
    name=vAR_TESTDATA+'\\'+i
    img = PIL.Image.open(name)
    result=ageneth_predictor.predict(img)
    with open(vAR_MODELOUTCOME+'\\'+'PATIENT SEGMENTATION.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i,result[0],result[1],result[2]]) 
    